# IronHacks Submission Notebook

## Background
---

Write a brief summary about goal of this notebook and provide some insight to the reader about how read through your document.

## Setup
---

In [40]:

%logstop
%logstart -t -r -q ipython_command_log.py global

#- IRONHACKS RESEARCH TRACKING CODE
#----------------------------------
# The following code is used to help our research team understand how you 
# our notebook environment. We do not collect any personal information with
# the following code, it is used to measure when and how often you work on
# your submission files.

import os
from datetime import datetime
import IPython.core.history as history

ha = history.HistoryAccessor()
ha_tail = ha.get_tail(1)
ha_cmd = next(ha_tail)
session_id = str(ha_cmd[0])
command_id = str(ha_cmd[1])
timestamp = datetime.utcnow().isoformat()
history_line = ','.join([session_id, command_id, timestamp]) + '\n'
logfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')
logfile.write(history_line)
logfile.close()

In [19]:


#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED
#------------------------------------------
# This is normally not required. The hub environment comes preinstaled with 
# many packages that you can already use without setup. In case there is some
# other library you would like to use that isn't on the list you run this command
# once to install them.  If it is already installed this command has no effect.
!python3 -m pip install google.cloud
!python3 -m pip install numpy
!python3 -m pip install pyarrow
!python3 -m pip install pandas

### Imports

In [20]:
#- IMPORT THE LIBRARIES YOU WILL USE
#------------------------------------------
# You only need to import packages one time per notebook session. To keep your
# notebook clean and organized you can handle all imports at the top of your file.
# The following are included for example purposed, feel free to modify or delete 
# anything in this section.

import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
magics.context.use_bqstorage_api = True
import pyarrow
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, linewidth=200, edgeitems=100)

# Pipeline and column transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

# Data transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, label_binarize
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Data splitter and model evaluator
from sklearn.model_selection import train_test_split, validation_curve, learning_curve, GridSearchCV

# Learning models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, GradientBoostingClassifier, GradientBoostingRegressor
#from xgboost import XGBClassifier, XGBRegressor  # Need to install

# Performance metrics
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [21]:
# CONFIGURE THE BIGQUERY SETTINGS

BIGQUERY_PROJECT = 'ironhacks-covid19-data'
BIGQUERY_KEYPATH = 'service-account.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

## Classes and Functions
---

In [31]:
#- DEFINE YOUR CLASSES AND FUNCTIONS 
#-----------------------------------
# This is not required, but is helpful in keeping your notebook organized. 
# You can use the following cell or several cells to define your functions
# and classes to keep them separate from your analysis or results code.
# In general it useful to define your methods in a separate cell from where
# it is run.

def example_function():
    print('Hello World')

In [32]:
# This function is to get the data I need from the big query

def get_data():
    query = """
    SELECT poi_id, top_category, week_number, raw_visit_counts, postal_code
    FROM ironhacks_covid19_competition.weekly_patterns
    """
    query_job = bigquery_client.query(query)
    wp = query_job.to_dataframe()

    query = """
    SELECT week_number, cases
    FROM ironhacks_covid19_competition.covid19_cases
    """
    query_job = bigquery_client.query(query)
    covid = query_job.to_dataframe()
    return wp, covid
    

In [33]:
#shifting the covid_cases up one and two weeks to decrease the delay in collecting data and people's reaction towards the coivd cases

def add_covid_cases(covid, wp):
    nc = covid.copy()
    nc['cases_shift1'] = nc['cases'].shift(1)
    nc['cases_shift2'] = nc['cases'].shift(2)
    del nc['cases']
    nc = nc.fillna(7)
    new = wp.merge(nc, on = 'week_number')
    return new

In [34]:
# Changing the format of the table to having all the categories, poi_id, and postal_code as column headders
# The fillers will be containing 0 or 1 depending on the values in week_number and raw_visit_counts

def change_format(new):
    dummy = pd.get_dummies(new['top_category'])
    table = new.join(dummy)
    dummy = pd.get_dummies(new['poi_id'])
    table = table.join(dummy)
    dummy = pd.get_dummies(new['postal_code'])
    table = table.join(dummy)
    del table['top_category']
    del table['poi_id']
    del table['postal_code']
    return table

In [35]:
# Here I'm making my predictinon table which contains all the known informations for week 44 except for raw_visit_counts

def prediction_data(new):
    filt_wp = new[new['week_number'] == 43]
    dummy = pd.get_dummies(filt_wp['top_category'])
    table2 = filt_wp.join(dummy)
    dummy = pd.get_dummies(filt_wp['poi_id'])
    table2 = table2.join(dummy)
    dummy = pd.get_dummies(filt_wp['postal_code'])
    table2 = table2.join(dummy)
    del table2['top_category']
    del table2['raw_visit_counts']
    del table2['postal_code']

    table2['week_number'] = table2['week_number'].replace(43, 44)
    table2['cases_shift1'] = table2['cases_shift1'].replace(25872, 29641)
    table2['cases_shift2'] = table2['cases_shift2'].replace(23218, 25872)
    return table2

In [36]:
# Making a decision tree model based on the table we've given to it
# I set raw_visit_counts as the Y value and everything else is X
# Also splited the data randomly into 80% for training set and 20% for testing
# Prints out the training score and testing score


def decision_tree(table):
    df = table
    X = df.drop(['raw_visit_counts'], axis=1)
    y = df['raw_visit_counts']
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0)

    decision_tree_model = DecisionTreeRegressor(random_state=0)
    decision_tree_model.fit(X_train, y_train)
    print('Decision Tree Training score:', decision_tree_model.score(X_train,y_train))
    print('Decision Tree Test score:    ', decision_tree_model.score(X_test,y_test),'\n')
    return decision_tree_model

In [37]:
# Predicts the week 44's raw_visit_counts given the trained model and the week 44 known data

def presict_week_44(table2, decision_tree_model):
    X_predict = table2
    poi_id = table2['poi_id']
    del X_predict['poi_id']
    raw_visit_counts = decision_tree_model.predict(X_predict)


    d = pd.DataFrame(columns = ['poi_id', 'raw_visit_counts'])
    d['poi_id'] = poi_id
    d['raw_visit_counts'] = raw_visit_counts
    return d

## Analysis
---

In [38]:
# Run the functions and store the predicted result in the variable called result

example_function()
wp,covid = get_data()
combined = add_covid_cases(covid, wp)
table = change_format(combined)
table2 = prediction_data(combined)
decision_tree_model = decision_tree(table)
result = presict_week_44(table2, decision_tree_model)


Hello World


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Decision Tree Training score: 1.0
Decision Tree Test score:     0.9581896310451774 



## Results
---

This section may not be necessary for you individual notebook, but if you've created nice charts or tables you can present them together with some explanation in section. In addition if you have any comments or thoughts about what you would like to do or items that are still in progress you can summarize them at the bottom of the page.

In [30]:
## This can also be a good place for you to cleanup any input/output and export your results to a file.
# Storing the result into the csv file
result.to_csv('submission_prediction_output.csv',index=False)